In [1]:
!pip install -q -U kaggle_environments

     |████████████████████████████████| 1.9 MB 5.2 MB/s 
     |████████████████████████████████| 62 kB 1.5 MB/s 


In [2]:
import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
import seaborn as sns

from kaggle_environments import make, evaluate

Агент всегда играет камень.

In [3]:
%%writefile rock_agent.py

def rock_agent(observation, configuration):
    return 0

Writing rock_agent.py


Агент всегда играет бумага.

In [4]:
%%writefile paper_agent.py

def paper_agent(observation, configuration):
    return 1


Writing paper_agent.py


Агент всегда играет ножницы.

In [5]:
%%writefile scissors_agent.py

def scissors_agent(observation, configuration):
    return 2

Writing scissors_agent.py


Агент повторяет прошлые действия оппонента. 

In [6]:
%%writefile copy_agent.py

import random
def copy_agent(observation, configuration):
    # Получает прошлое действие оппонента
    if observation.step > 0:
        return observation.lastOpponentAction
    # Первый шаг оппонента - случайное значение
    else:
        return random.randrange(0, configuration.signs)

Writing copy_agent.py


Агент возвращает действие, которое победит самое популярное из прошлых действий оппонента. 

In [7]:
%%writefile popular_agent.py

# Прошлые действия оппоннета
opponent_actions = []

def popular_agent(observation, configuration):
    global opponent_actions
    
    count = len(opponent_actions)
    
    # Первый шаг оппонента - случайное значение
    if count == 0:
        opponent_actions.append(random.randrange(0, configuration.signs)) 
    # Находим самое используемое действие опонента из прошлых
    else:
        opponent_actions.append(observation.lastOpponentAction)
        action_count = [opponent_actions.count(0), opponent_actions.count(1), opponent_actions.count(2)]        
        popular_opponent_action = action_count.index(max(action_count))

        if popular_opponent_action == 0: 
            return 1
        elif popular_opponent_action == 1:
            return 2
        else:
            return 0

Writing popular_agent.py


Агент при победе в прошлой игре возвращает прошлое действие, при проигрыше или игре в ничью меняет действие на случайное, за исключением невыигрышного действия.

In [23]:
%%writefile last_agent.py

import random
last_my_action = None

def last_agent(observation, configuration):
    global last_my_action

    # Определение исхода игры
    def get_result(last_my_action, last_opponent_action):
        if last_my_action == last_opponent_action:
            return 0
        elif (last_my_action == 1 and last_opponent_action == 0) \
            or (last_my_action == 0 and last_opponent_action == 1) \
            or (last_my_action == 2 and last_opponent_action == 1):
            return 1
        else:
            return -1    

    if observation.step == 0:
    # Первый шаг - камень
        last_opponent_action = 0 
        last_my_action = 0
    else:
        last_opponent_action = observation.lastOpponentAction
    result = get_result(last_my_action, last_opponent_action)

    if result != 1: 
        # Если в прошлой игре не было победы, меняем действие на случайное, за исключением невыигрышного действия
        list_of_action = [action for action in [0, 1, 2] if action != last_my_action] 
        last_my_action = random.choice(list_of_action)       
    return last_my_action
    

Overwriting last_agent.py


Агент возвращает случайное значение. 

In [9]:
%%writefile random_agent.py

import random

def random_agent(observation, configuration):
    return random.randint(0, 2)

Writing random_agent.py


Агент, который возвращает свои предыдущие шаги cо сдвигом.

In [10]:
%%writefile my_previous_agent.py

# Действие прошлых игр
last_games = []

def my_previous_agent(observation, configuration):
    global last_games

    if observation.step == 0:
    # Первый шаг - камень
        agent = 0
        last_games.append(agent)
    else:
        agent = (last_games[-1] + 1) % 3 
        last_games.append(agent)
    return agent

Writing my_previous_agent.py


Абонент, который возвращает предыдущие шаги оппонента cо сдвигом

In [11]:
%%writefile opponent_previous_agent.py


# Действие оппонента в прошлых играх
last_games = []

def opponent_previous_agent(observation, configuration):
    global last_games

    if observation.step == 0:
    # Первый шаг оппонента - камень    
        opponent_action = 0
        last_games.append(opponent_action)
    else:
        last_games[-1] = observation.lastOpponentAction
        opponent_action = (last_games[-1] + 1) % 3
        last_games.append(opponent_action)
    return opponent_action

Writing opponent_previous_agent.py


Абонент возвращает камень, если шаг четный, случайное значение среди ножниц и бумаги - если шаг нечетный.

In [12]:
%%writefile even_agent.py

import random
def even_agent(observation, configuration):  
    if observation.step % 2 == 0:
        return 0
    else:
        return random.choice([1, 2])

Writing even_agent.py


Агент возвращает действие, которое победит самое популярное из прошлых действий оппонента, если вероятность выпадания самого популярного действия оппонета больше 20%. 
В противном случае агент возвращает самое популярное из прошлых действий оппонента.

In [13]:
%%writefile probability_agent.py

import random

# Прошлые действия оппонента
opponent_actions = []

def probability_agent(observation, configuration):
    global opponent_actions
    
    count = len(opponent_actions) # Количество прошлых действий 
    
    # Первый шаг оппонента - случайное значение.
    if count == 0:
        opponent_actions.append(random.randrange(0, configuration.signs)) 
    # Находим самое используемое действие опонента из прошлых.
    else:
        opponent_actions.append(observation.lastOpponentAction)
        action_count = [opponent_actions.count(0), opponent_actions.count(1), opponent_actions.count(2)]        
        popular_opponent_action = action_count.index(max(action_count))
        # Вероятность выпадания популярного действия. 
        probability = popular_opponent_action / count 

        if probability > 20:
            if popular_opponent_action == 0: 
                return 1
            elif popular_opponent_action == 1:
                return 2
            else:
                return 0
        else:
            return popular_opponent_action

Writing probability_agent.py


Агент возвращает следующее по отношению к действиям оппонента действие. Порядок действий задан следующим образом: камень, бумага, ножницы.

In [14]:
%%writefile ordered_agent.py

import random

def ordered_agent(observation, configuration):
    if observation.step > 0:
        if observation.lastOpponentAction == 0:
            return 1
        elif observation.lastOpponentAction == 1:
            return 2
        else:
            return 0
    else:
        # Первый шаг - случайное действие
        return random.randrange(0, configuration.signs)

Writing ordered_agent.py


Агент при победе возвращает действие, которое не играло в прошлой игре, при проигрыше или игре в ничью - действие оппонента в прошлой игре. 

In [15]:
%%writefile thirteen_agent.py

# Действия игроков в прошлых играх
last_games = []

def thirteen_agent(observation, configuration):
    global last_games

    # Определение исхода игры
    def get_result(last_my_action, last_opponent_action):
        if last_my_action == last_opponent_action:
            return 0
        elif (last_my_action == 1 and last_opponent_action == 0) \
            or (last_my_action == 0 and last_opponent_action == 1) \
            or (last_my_action == 2 and last_opponent_action == 1):
            return 1
        else:
            return -1  

    if observation.step == 0:
    # Первый шаг - камень
        agent = 0
        last_games.append({'agent' : agent, 'opponent' : None})
        return agent
    else:
        last_games[-1]['opponent'] = observation.lastOpponentAction
        result = get_result(last_games[-1]['agent'], last_games[-1]['opponent']) 
        
        if result == 1: 
            agent = [action for action in [0, 1, 2] if action != last_games[-1]['agent'] and action != last_games[-1]['opponent']] 
            last_games.append({'agent' : agent[0], 'opponent': None})
        else: 
            agent = last_games[-1]['opponent'] 
            last_games.append({'agent' : agent, 'opponent': None})
        return last_games[-2]['agent']
            

Writing thirteen_agent.py


Воспользуемся функцией evaluate из библиотеки kaggle_environments с помощью которой запустим наших агентов и проведем эксперимент на заданном количестве игр

In [30]:
evaluate(
    "rps", 
    ["last_agent.py", "thirteen_agent.py"], # Агенты, которые будут играть
    configuration={"episodeSteps": 100} # Количество игр 
)

[[-21.0, 21.0]]